In [1]:
import numpy as np
from collections import Counter
from sklearn.datasets import load_iris
from graphviz import Digraph

class Question:
    def __init__(self, column, value, feature_name=None):
        self.column = column
        self.value = value
        self.feature_name = feature_name

    def match(self, example):
        val = example[self.column]
        if isinstance(val, int) or isinstance(val, float):
            return val >= self.value
        else:
            return val == self.value

    def __repr__(self):
        condition = "=="
        if isinstance(self.value, int) or isinstance(self.value, float):
            condition = ">="
        if self.feature_name:
            return f"Is {self.feature_name} {condition} {self.value}?"
        else:
            return f"Is feature {self.column} {condition} {self.value}?"

def partition(rows, question):
    true_rows, false_rows = [], []
    for row in rows:
        if question.match(row):
            true_rows.append(row)
        else:
            false_rows.append(row)
    return true_rows, false_rows

def entropy(rows):
    counts = class_counts(rows)
    impurity = 0
    total = len(rows)
    for label in counts:
        prob_of_label = counts[label] / total
        impurity -= prob_of_label * np.log2(prob_of_label)
    return impurity

def info_gain(left, right, current_entropy):
    p = float(len(left)) / (len(left) + len(right))
    return current_entropy - p * entropy(left) - (1 - p) * entropy(right)

class Leaf:
    def __init__(self, rows, target_names):
        counts = class_counts(rows)
        self.predictions = {target_names[int(k)]: v for k, v in counts.items()}
        self.entropy = entropy(rows)

class Decision_Node:
    def __init__(self, question, true_branch, false_branch, info_gain, entropy):
        self.question = question
        self.true_branch = true_branch
        self.false_branch = false_branch
        self.info_gain = info_gain
        self.entropy = entropy

def build_tree(rows, target_names):
    gain, question = find_best_split(rows)

    if gain == 0:
        return Leaf(rows, target_names)

    true_rows, false_rows = partition(rows, question)
    true_branch = build_tree(true_rows, target_names)
    false_branch = build_tree(false_rows, target_names)

    return Decision_Node(question, true_branch, false_branch, gain, entropy(rows))

def class_counts(rows):
    return dict(Counter(row[-1] for row in rows))

def find_best_split(rows):
    best_gain = 0
    best_question = None
    current_entropy = entropy(rows)
    n_features = len(rows[0]) - 1

    for col in range(n_features):
        values = set([row[col] for row in rows])

        for val in values:
            question = Question(col, val)
            true_rows, false_rows = partition(rows, question)

            if len(true_rows) == 0 or len(false_rows) == 0:
                continue

            gain = info_gain(true_rows, false_rows, current_entropy)

            if gain >= best_gain:
                best_gain, best_question = gain, question

    return best_gain, best_question

def print_tree(node, level=0):
    if isinstance(node, Leaf):
        print(f"Level {level}")
        for key, value in node.predictions.items():
            print(f"Count of {key} = {value}")
        print(f"Current Entropy is = {node.entropy:.12f}")
        print("Reached leaf Node")
        return

    print(f"Level {level}")
    rows = [list(row) for row in data]
    for key, value in class_counts(rows).items():
        print(f"Count of {key} = {value}")

    print(f"Current Entropy is = {entropy(rows):.12f}")
    feature_name = node.question.feature_name if node.question.feature_name else f"feature {node.question.column}"
    print(f"Splitting on {feature_name} ({node.question.value}) with gain ratio {node.info_gain:.12f}")

    print_tree(node.true_branch, level + 1)
    print_tree(node.false_branch, level + 1)

def visualize_tree(tree, filename):
    dot = Digraph(comment='Decision Tree')

    def add_nodes(node):
        if isinstance(node, Leaf):
            dot.node(str(id(node)), label=f"Predict\n{node.predictions}\nEntropy: {node.entropy:.4f}", shape='box')
        else:
            label = f"{node.question}\nInfo Gain: {node.info_gain:.4f}\nEntropy: {node.entropy:.4f}"
            dot.node(str(id(node)), label=label, shape='box')
            add_nodes(node.true_branch)
            add_nodes(node.false_branch)
            dot.edge(str(id(node)), str(id(node.true_branch)), label="True")
            dot.edge(str(id(node)), str(id(node.false_branch)), label="False")

    add_nodes(tree)
    dot.render(filename, format='pdf', cleanup=True)

if __name__ == "__main__":
    iris = load_iris()
    X = iris.data
    y = iris.target
    target_names = iris.target_names
    data = np.c_[X, y]

    tree = build_tree(data, target_names)
    print_tree(tree)
    visualize_tree(tree, "decision_tree1")


Level 0
Count of 0.0 = 50
Count of 1.0 = 50
Count of 2.0 = 50
Current Entropy is = 1.584962500721
Splitting on feature 3 (1.0) with gain ratio 0.918295834054
Level 1
Count of 0.0 = 50
Count of 1.0 = 50
Count of 2.0 = 50
Current Entropy is = 1.584962500721
Splitting on feature 3 (1.8) with gain ratio 0.690160370755
Level 2
Count of 0.0 = 50
Count of 1.0 = 50
Count of 2.0 = 50
Current Entropy is = 1.584962500721
Splitting on feature 2 (4.9) with gain ratio 0.091208111774
Level 3
Count of virginica = 43
Current Entropy is = 0.000000000000
Reached leaf Node
Level 3
Count of 0.0 = 50
Count of 1.0 = 50
Count of 2.0 = 50
Current Entropy is = 1.584962500721
Splitting on feature 1 (3.2) with gain ratio 0.918295834054
Level 4
Count of versicolor = 1
Current Entropy is = 0.000000000000
Reached leaf Node
Level 4
Count of virginica = 2
Current Entropy is = 0.000000000000
Reached leaf Node
Level 2
Count of 0.0 = 50
Count of 1.0 = 50
Count of 2.0 = 50
Current Entropy is = 1.584962500721
Splitting on 